In [1]:
import numpy as np
import twixtools
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
#filename = "/mnt/c/Users/giuse/Downloads/meas_MID00195_FID07959_T1_Sag_MPRAGE_p2_TR_1870_0_8mm.dat"
filename = "/mnt/c/Users/giuse/Downloads/MR160-088-0539-02-RAW/meas_MID00053_FID41387_T1_Sag_MPRAGE_p2_TR_1870_0_8mm.dat"
multi_twix = twixtools.read_twix(filename)

Software version: VD/VE (!?)

Scan  0


100%|██████████████████████████████████████████████████████████████████████████████| 38.1M/38.1M [00:00<00:00, 44.6MB/s]


Scan  1


 72%|███████████████████████████████████████████████████████▉                      | 2.67G/3.72G [10:23<04:28, 4.21MB/s]

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 0: ordinal not in range(128)

# list the mdh flags and line counters for every 8th measurement data block (mdb)
for mdb in multi_twix[-1]['mdb'][::8]:
    print('line: %3d; flags:'%(mdb.cLin), mdb.get_active_flags())

In [ ]:
def ifftnd(kspace, axes=[-1]):
    from numpy.fft import fftshift, ifftshift, ifftn
    if axes is None:
        axes = range(kspace.ndim)
    img = fftshift(ifftn(ifftshift(kspace, axes=axes), axes=axes), axes=axes)
    img *= np.sqrt(np.prod(np.take(img.shape, axes)))
    return img


def rms_comb(sig, axis=1):
    return np.sqrt(np.sum(abs(sig)**2, axis))

In [ ]:
image_mdbs = [mdb for mdb in multi_twix[-1]['mdb'] if mdb.is_image_scan()]


# sort all 'imaging' mdbs into a k-space array
image_mdbs = [mdb for mdb in multi_twix[-1]['mdb'] if mdb.is_image_scan()]

for mdb in image_mdbs:
    print(mdb.cLin)

n_line = 1 + max([mdb.cLin for mdb in image_mdbs])

# assume that all data were acquired with same number of channels & columns:
n_channel, n_column = image_mdbs[0].data.shape

kspace = np.zeros([n_line, n_channel, n_column], dtype=np.complex64)
for mdb in image_mdbs:
    kspace[mdb.cLin] = mdb.data

print('\nk-space shape', kspace.shape)
print(kspace[:,0].shape)

kspace = np.fft.fftshift(kspace)
# reconstruct an image and show the result:
plt.figure(figsize=[12,8])
plt.subplot(121)
plt.title('k-space')
plt.imshow(abs(kspace[:,0]), cmap='gray', origin='lower')
plt.axis('off')



image = ifftnd(kspace, [0,-1])
image = rms_comb(image)
plt.subplot(122)
plt.title('image')
plt.imshow(abs(image), cmap='gray', origin='lower')
plt.axis('off')

fig, axes = plt.subplots(5, 4, figsize=(14,14), sharex=True, sharey=True)

for row in range(5):
    for column in range(4):
        ax = axes[row, column]
        coil = row * 4 + column
        ax.set_title(f"Coil# {coil+1}")
        ax.axis('off')
        ax.imshow(np.abs(kspace[:,coil])**0.2, cmap='gray', origin='lower')
plt.tight_layout()
plt.show()

In [ ]:
# read image data from list of mdbs and sort into 3d k-space (+ coil dim.)
def import_kspace(mdb_list):
    image_mdbs = []
    for mdb in mdb_list:
        if mdb.is_image_scan():
            image_mdbs.append(mdb)

    n_line = 1 + max([mdb.cLin for mdb in image_mdbs])
    n_part = 1 + max([mdb.cPar for mdb in image_mdbs])
    n_channel, n_column = image_mdbs[0].data.shape

    out = np.zeros([n_part, n_line, n_channel, n_column], dtype=np.complex64)
    for mdb in image_mdbs:
        # '+=' takes care of averaging, but careful in case of other counters (e.g. echoes)
        out[mdb.cPar, mdb.cLin] += mdb.data

    return out  # 4D numpy array [n_part, n_line, n_channel, n_column]

output = import_kspace(multi_twix[-1]['mdb'])

In [ ]:
# map the twix data to twix_array objects
mapped = twixtools.map_twix(multi_twix)
im_data = mapped[-1]['image']

# make sure that we later squeeze the right dimensions:
print(im_data.non_singleton_dims)

# the twix_array object makes it easy to remove the 2x oversampling in read direction
im_data.flags['remove_os'] = True

# read the data (array-slicing is also supported)
data = im_data[:].squeeze()


In [ ]:
data = im_data[:].squeeze()

In [ ]:
image = ifftnd(output, [0, 1,-1])
image = rms_comb(image, axis=-2)

In [ ]:
output.shape
#plt.imshow(np.abs([0,:,0]), cmap='gray')